In [1]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.plotly as py
import cufflinks as cf
cf.go_offline()
from ipywidgets import interact

from scipy import stats
import scipy.stats as stats
import statsmodels as sm

In [22]:
# primary types
# location description
# arrest rate/geolocation
# remove columns
# oversample target variable
# outliers
# normalize the data
# choose regressor
# features scale 
# grid search for besrt hyperparameters
# cross-validation
# severity?

In [16]:
pd.set_option('display.max_columns',100)

In [2]:
chicago_cmr = pd.read_csv('/Users/adrientalbot/Documents/final-project/data/chicago_clean.csv')

In [3]:
chicago_formatted = chicago_cmr.drop(columns = ['Latitude','Longitude'])

In [4]:
chicago_formatted.head()

,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Year,Location
0,2001-01-01 00:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,631,6.0,2001,"(41.748623027, -87.608654109)"
1,2001-01-01 00:00:00,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,0,0,624,6.0,2001,"(41.754802305, -87.598146873)"
2,2001-01-01 00:00:00,0810,THEFT,OVER $500,APARTMENT,0,0,2332,19.0,2001,"(41.938952432, -87.638163582)"
3,2001-01-01 00:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,624,6.0,2001,"(41.752061911, -87.607827503)"
4,2001-01-01 00:00:00,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,0,0,1013,10.0,2001,"(41.850385805, -87.715107802)"


# One hot encoding for Primary Types

In [9]:
primary_type_count = chicago_formatted['Primary Type'].value_counts()
df_primary_type_count = pd.DataFrame(primary_type_count).reset_index().rename({"index": "primary type", "Primary Type": "count"}, axis = 1)
top_23_primary = df_primary_type_count[df_primary_type_count['count'] > 5000]
top_23_primary_lst = list(top_23_primary['primary type'].unique())

In [25]:
primary_type_count

THEFT                                1425163
BATTERY                              1244942
CRIMINAL DAMAGE                       777380
NARCOTICS                             707206
ASSAULT                               424171
OTHER OFFENSE                         422279
BURGLARY                              390379
MOTOR VEHICLE THEFT                   314879
DECEPTIVE PRACTICE                    257647
ROBBERY                               257343
CRIMINAL TRESPASS                     195041
WEAPONS VIOLATION                      72432
PROSTITUTION                           67919
PUBLIC PEACE VIOLATION                 48135
OFFENSE INVOLVING CHILDREN             43994
CRIM SEXUAL ASSAULT                    26408
SEX OFFENSE                            24384
INTERFERENCE WITH PUBLIC OFFICER       15574
GAMBLING                               14340
LIQUOR LAW VIOLATION                   14002
ARSON                                  11210
HOMICIDE                                9608
KIDNAPPING

In [10]:
lst_primary_types = ['THEFT', 'OFFENSE INVOLVING CHILDREN', 'ASSAULT',
       'CRIM SEXUAL ASSAULT', 'SEX OFFENSE', 'DECEPTIVE PRACTICE',
       'CRIMINAL DAMAGE', 'BATTERY',
       'MOTOR VEHICLE THEFT', 'BURGLARY', 'WEAPONS VIOLATION',
       'CRIMINAL TRESPASS', 'ROBBERY', 'NARCOTICS',
       'LIQUOR LAW VIOLATION', 'PUBLIC PEACE VIOLATION',
       'INTERFERENCE WITH PUBLIC OFFICER', 'HOMICIDE', 'GAMBLING',
       'KIDNAPPING', 'PROSTITUTION', 'ARSON']

In [11]:
chicago_formatted_primary = chicago_formatted[chicago_formatted['Primary Type'].isin(lst_primary_types)]

In [13]:
for primary_type in lst_primary_types:
    chicago_formatted_primary[primary_type] = 0

In [18]:
for primary_type in lst_primary_types:
    chicago_formatted_primary[primary_type] = np.where(chicago_formatted_primary['Primary Type'].str.contains(primary_type), 1, 0)


In [19]:
chicago_formatted_primary.head()

,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Year,Location,THEFT,OFFENSE INVOLVING CHILDREN,ASSAULT,CRIM SEXUAL ASSAULT,SEX OFFENSE,DECEPTIVE PRACTICE,CRIMINAL DAMAGE,BATTERY,MOTOR VEHICLE THEFT,BURGLARY,WEAPONS VIOLATION,CRIMINAL TRESPASS,ROBBERY,NARCOTICS,LIQUOR LAW VIOLATION,PUBLIC PEACE VIOLATION,INTERFERENCE WITH PUBLIC OFFICER,HOMICIDE,GAMBLING,KIDNAPPING,PROSTITUTION,ARSON
0,2001-01-01 00:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,631,6.0,2001,"(41.748623027, -87.608654109)",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2001-01-01 00:00:00,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,0,0,624,6.0,2001,"(41.754802305, -87.598146873)",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2001-01-01 00:00:00,0810,THEFT,OVER $500,APARTMENT,0,0,2332,19.0,2001,"(41.938952432, -87.638163582)",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2001-01-01 00:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,624,6.0,2001,"(41.752061911, -87.607827503)",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2001-01-01 00:00:00,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,0,0,1013,10.0,2001,"(41.850385805, -87.715107802)",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
chicago_formatted_primary['Primary Type'].unique()

array(['THEFT', 'OFFENSE INVOLVING CHILDREN', 'ASSAULT',
       'CRIM SEXUAL ASSAULT', 'SEX OFFENSE', 'DECEPTIVE PRACTICE',
       'CRIMINAL DAMAGE', 'BATTERY', 'MOTOR VEHICLE THEFT', 'BURGLARY',
       'WEAPONS VIOLATION', 'CRIMINAL TRESPASS', 'ROBBERY', 'NARCOTICS',
       'LIQUOR LAW VIOLATION', 'PUBLIC PEACE VIOLATION',
       'INTERFERENCE WITH PUBLIC OFFICER', 'HOMICIDE', 'GAMBLING',
       'KIDNAPPING', 'PROSTITUTION', 'ARSON'], dtype=object)

# One encoding for location description

In [23]:
location_count = chicago_formatted_primary['Location Description'].value_counts()

In [30]:
df_location_count = pd.DataFrame(location_count).reset_index().rename({"index": "location description", "Location Description": "count"}, axis = 1)
top_23_location = df_location_count[df_location_count['count'] > 35000]
top_23_location_lst = list(top_23_location['location description'].unique())

In [32]:
lst_location = ['STREET',
 'RESIDENCE',
 'SIDEWALK',
 'APARTMENT',
 'PARKING LOT/GARAGE(NON.RESID.)',
 'ALLEY',
 'SCHOOL, PUBLIC, BUILDING',
 'RESIDENCE-GARAGE',
 'SMALL RETAIL STORE',
 'RESIDENCE PORCH/HALLWAY',
 'RESTAURANT',
 'VEHICLE NON-COMMERCIAL',
 'GROCERY FOOD STORE',
 'DEPARTMENT STORE',
 'GAS STATION',
 'RESIDENTIAL YARD (FRONT/BACK)',
 'CHA PARKING LOT/GROUNDS',
 'PARK PROPERTY',
 'COMMERCIAL / BUSINESS OFFICE',
 'CTA PLATFORM',
 'BAR OR TAVERN']

In [33]:
chicago_formatted_primary_location = chicago_formatted_primary[chicago_formatted_primary['Location Description'].isin(lst_location)]

In [37]:
chicago_formatted_primary_location['Primary Type'].unique()

array(['THEFT', 'OFFENSE INVOLVING CHILDREN', 'ASSAULT',
       'CRIM SEXUAL ASSAULT', 'SEX OFFENSE', 'DECEPTIVE PRACTICE',
       'CRIMINAL DAMAGE', 'BATTERY', 'MOTOR VEHICLE THEFT', 'BURGLARY',
       'WEAPONS VIOLATION', 'CRIMINAL TRESPASS', 'ROBBERY', 'NARCOTICS',
       'LIQUOR LAW VIOLATION', 'PUBLIC PEACE VIOLATION', 'GAMBLING',
       'PROSTITUTION', 'KIDNAPPING', 'INTERFERENCE WITH PUBLIC OFFICER',
       'ARSON', 'HOMICIDE'], dtype=object)

In [39]:
for location in lst_location:
    chicago_formatted_primary_location[location] = 0

In [41]:
for location in lst_location:
    chicago_formatted_primary_location[location] = np.where(chicago_formatted_primary_location['Location Description'].str.contains(location), 1, 0)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



In [42]:
chicago_formatted_primary_location.head()

,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Year,Location,THEFT,OFFENSE INVOLVING CHILDREN,ASSAULT,CRIM SEXUAL ASSAULT,SEX OFFENSE,DECEPTIVE PRACTICE,CRIMINAL DAMAGE,BATTERY,MOTOR VEHICLE THEFT,BURGLARY,WEAPONS VIOLATION,CRIMINAL TRESPASS,ROBBERY,NARCOTICS,LIQUOR LAW VIOLATION,PUBLIC PEACE VIOLATION,INTERFERENCE WITH PUBLIC OFFICER,HOMICIDE,GAMBLING,KIDNAPPING,PROSTITUTION,ARSON,STREET,RESIDENCE,SIDEWALK,APARTMENT,PARKING LOT/GARAGE(NON.RESID.),ALLEY,"SCHOOL, PUBLIC, BUILDING",RESIDENCE-GARAGE,SMALL RETAIL STORE,RESIDENCE PORCH/HALLWAY,RESTAURANT,VEHICLE NON-COMMERCIAL,GROCERY FOOD STORE,DEPARTMENT STORE,GAS STATION,RESIDENTIAL YARD (FRONT/BACK),CHA PARKING LOT/GROUNDS,PARK PROPERTY,COMMERCIAL / BUSINESS OFFICE,CTA PLATFORM,BAR OR TAVERN
0,2001-01-01 00:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,631,6.0,2001,"(41.748623027, -87.608654109)",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2001-01-01 00:00:00,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,0,0,624,6.0,2001,"(41.754802305, -87.598146873)",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2001-01-01 00:00:00,0810,THEFT,OVER $500,APARTMENT,0,0,2332,19.0,2001,"(41.938952432, -87.638163582)",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2001-01-01 00:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,624,6.0,2001,"(41.752061911, -87.607827503)",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2001-01-01 00:00:00,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,0,0,1013,10.0,2001,"(41.850385805, -87.715107802)",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
one_hot_table = chicago_formatted_primary_location.drop(['Primary Type', 'Description','Location Description', 'Beat', 'Year'], axis = 1)

# Add arrest rate/geolocation

# Normalize data